# 🏛️ Fase 1: Pelatihan Model Klasifikasi PCB

**Tujuan:** Melatih model klasifikasi gambar untuk deteksi defect PCB (Lulus QC vs. Cacat Produksi)

**Output:** File model Keras (.h5) yang berkinerja tinggi

---

## ⚙️ Setup Lingkungan

### **📍 Opsi 1: Google Colab (Gratis GPU - Recommended untuk Pemula)**

**Langkah-langkah:**
1. Buka Google Colab: https://colab.research.google.com
2. Upload notebook ini (File → Upload notebook)
3. Ubah runtime ke **T4 GPU**: Runtime > Change runtime type > T4 GPU
4. Jalankan sel-sel di bawah secara berurutan

**Kelebihan:**
- ✅ Gratis GPU (T4)
- ✅ Tidak perlu install apapun
- ✅ Akses dari browser

---

### **📍 Opsi 2: Local dengan GPU (RTX 3080 Ti atau GPU lainnya)**

**Langkah-langkah:**
1. Install dependencies:
   ```bash
   pip install -r requirements.txt
   ```

2. Setup CUDA & cuDNN (untuk GPU):
   - NVIDIA Driver terbaru
   - CUDA Toolkit 11.8
   - cuDNN 8.6

3. Jalankan Jupyter Notebook:
   ```bash
   jupyter notebook
   ```

4. Buka file ini dan jalankan cell-cell di bawah

**Kelebihan:**
- ✅ Sangat cepat (mixed precision di RTX 30xx)
- ✅ Tidak terbatas waktu
- ✅ Privacy (data lokal)
- ✅ Kontrol penuh

---

### **💡 Catatan:**
- Cell upload dataset (dengan `files.upload()`) hanya berfungsi di Google Colab
- Jika di local, skip cell upload dan pastikan folder `dataset/` sudah ada di direktori yang sama
- Semua cell lainnya kompatibel untuk Colab dan Local!

In [ ]:
# Verifikasi GPU tersedia
import tensorflow as tf
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Set memory growth untuk GPU (mencegah OOM errors)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(e)

## 📦 Upload & Ekstrak Dataset

**Struktur yang diharapkan:**
```
dataset/
├── lulus_qc/          # Gambar PCB yang lolos QC
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
└── cacat_produksi/    # Gambar PCB dengan defect
    ├── image1.jpg
    ├── image2.jpg
    └── ...
```

In [ ]:
# ⚠️ CATATAN: Cell ini hanya untuk Google Colab!
# Jika menjalankan di local, skip cell ini dan pastikan folder 'dataset/' sudah ada

# Upload dataset.zip dari komputer lokal (Google Colab only)
try:
    from google.colab import files
    import os

    print("📤 Silakan upload file dataset.zip Anda...")
    uploaded = files.upload()

    # Ekstrak dataset
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            print(f"\n📂 Mengekstrak {filename}...")
            !unzip -q {filename}
            print("✅ Ekstraksi selesai!")
            
except ImportError:
    print("⚠️ Google Colab tidak terdeteksi.")
    print("💡 Pastikan folder 'dataset/' sudah ada di direktori yang sama dengan notebook ini.")
    print("   Struktur harus: dataset/lulus_qc/ dan dataset/cacat_produksi/")
    
    import os
    if os.path.exists('dataset'):
        print("✅ Folder 'dataset/' ditemukan!")
    else:
        print("❌ Folder 'dataset/' tidak ditemukan!")
        print("   Buat folder 'dataset/' dan tambahkan subfolder kelas Anda.")

In [ ]:
# Verifikasi struktur folder dataset
import os
from pathlib import Path

DATASET_DIR = 'dataset'

print("📁 Struktur Dataset:")
print("=" * 50)

if os.path.exists(DATASET_DIR):
    for root, dirs, files in os.walk(DATASET_DIR):
        level = root.replace(DATASET_DIR, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        
        # Hitung jumlah file per folder
        if level == 1:  # Folder kelas
            image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            print(f"{indent}  └─ {len(image_files)} gambar")
            
    print("\n✅ Dataset ditemukan!")
else:
    print("❌ Folder 'dataset' tidak ditemukan!")
    print("\n💡 Pastikan struktur folder Anda adalah:")
    print("   dataset/lulus_qc/ dan dataset/cacat_produksi/")

## 📚 Import Library & Definisi Parameter

In [ ]:
# Import semua library yang diperlukan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

print("✅ Semua library berhasil diimport")

In [ ]:
# Parameter Training
IMG_SIZE = 224        # Ukuran input untuk MobileNetV2
BATCH_SIZE = 32       # Batch size untuk training
EPOCHS = 10          # Jumlah epoch (bisa ditambah jika perlu)
LEARNING_RATE = 0.001 # Learning rate
VALIDATION_SPLIT = 0.2  # 20% data untuk validasi

# Path
DATASET_DIR = 'dataset'
MODEL_SAVE_PATH = 'qc_inspector_model.h5'

print("📋 Parameter Training:")
print(f"   - Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"   - Batch Size: {BATCH_SIZE}")
print(f"   - Epochs: {EPOCHS}")
print(f"   - Learning Rate: {LEARNING_RATE}")
print(f"   - Validation Split: {VALIDATION_SPLIT * 100}%")

## 🔄 Persiapan Data (Data Loaders)

**Penting:** Menggunakan `preprocess_input` khusus dari MobileNetV2 untuk normalisasi gambar yang tepat.

In [ ]:
# Data Generator untuk Training (dengan augmentasi)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # KRITIS: Preprocessing MobileNetV2
    validation_split=VALIDATION_SPLIT,
    
    # Augmentasi data untuk meningkatkan generalisasi
    rotation_range=20,           # Rotasi random ±20 derajat
    width_shift_range=0.2,       # Geser horizontal 20%
    height_shift_range=0.2,      # Geser vertical 20%
    shear_range=0.2,            # Shear transformation
    zoom_range=0.2,             # Zoom in/out 20%
    horizontal_flip=True,        # Flip horizontal random
    fill_mode='nearest'          # Fill untuk pixel kosong
)

# Data Generator untuk Validation (tanpa augmentasi, hanya preprocessing)
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VALIDATION_SPLIT
)

print("✅ Data generators berhasil dibuat")

In [ ]:
# Training Data Loader
train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification
    subset='training',
    shuffle=True,
    seed=42
)

# Validation Data Loader
validation_generator = validation_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"\n📊 Dataset Summary:")
print(f"   - Training samples: {train_generator.samples}")
print(f"   - Validation samples: {validation_generator.samples}")
print(f"   - Total batches per epoch: {len(train_generator)}")

## 🏷️ Verifikasi Label (KRITIS!)

**SANGAT PENTING:** Catat urutan label ini untuk digunakan di Fase 2 & 3!

In [ ]:
# Verifikasi label mapping
print("🏷️ Label Mapping (CLASS INDICES):")
print("=" * 50)
print(train_generator.class_indices)
print("=" * 50)

# Simpan ke variabel untuk referensi
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

print("\n📝 Interpretasi:")
for class_name, index in class_indices.items():
    print(f"   - {class_name} → Label {index}")

print("\n⚠️ CATAT INFORMASI INI! Anda akan membutuhkannya untuk:")
print("   - Fase 2: Verifikasi prediksi Python")
print("   - Fase 3: Konversi TFLite & deployment")

In [ ]:
# Visualisasi sample gambar dari dataset
import matplotlib.pyplot as plt

def show_sample_images(generator, num_images=8):
    """Menampilkan sample gambar dari generator"""
    # Ambil satu batch
    images, labels = next(generator)
    
    # Reverse class indices untuk mendapatkan nama kelas
    reverse_indices = {v: k for k, v in generator.class_indices.items()}
    
    plt.figure(figsize=(15, 8))
    for i in range(min(num_images, len(images))):
        plt.subplot(2, 4, i + 1)
        
        # Denormalisasi gambar untuk visualisasi
        img = images[i]
        img = (img - img.min()) / (img.max() - img.min())
        
        plt.imshow(img)
        plt.title(f"Label: {reverse_indices[int(labels[i])]}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

print("🖼️ Sample gambar dari Training Set:")
show_sample_images(train_generator)

## 🏗️ Membangun Model (Transfer Learning)

Menggunakan **MobileNetV2** sebagai base model (pre-trained pada ImageNet)

In [ ]:
# Load MobileNetV2 pre-trained model (tanpa top layer)
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,  # Tidak termasuk fully connected layer
    weights='imagenet'  # Gunakan weights pre-trained dari ImageNet
)

# Freeze base model layers (tidak akan dilatih)
base_model.trainable = False

print(f"✅ Base Model (MobileNetV2) berhasil dimuat")
print(f"   - Total layers: {len(base_model.layers)}")
print(f"   - Trainable: {base_model.trainable}")

In [ ]:
# Tambahkan custom classification head
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Base model
x = base_model(inputs, training=False)  # training=False agar BatchNorm tidak update

# Custom head
x = GlobalAveragePooling2D()(x)         # Pooling untuk reduce dimensi
x = Dropout(0.3)(x)                     # Dropout untuk regularisasi (mencegah overfitting)
outputs = Dense(1, activation='sigmoid')(x)  # Output layer untuk binary classification

# Buat model final
model = Model(inputs, outputs)

print("✅ Model architecture berhasil dibuat")
print(f"\n📊 Model Summary:")

In [ ]:
# Tampilkan model summary
model.summary()

## 🎯 Kompilasi & Training Model

In [ ]:
# Kompilasi model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',  # Loss untuk binary classification
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

print("✅ Model berhasil dikompilasi")

In [ ]:
# Setup callbacks untuk training
callbacks = [
    # Early Stopping: Stop jika validation loss tidak membaik
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Model Checkpoint: Simpan model terbaik
    ModelCheckpoint(
        'best_model_checkpoint.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    # Reduce Learning Rate: Kurangi LR jika stuck
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Training callbacks berhasil dikonfigurasi")

In [ ]:
# Mulai training
print("🚀 Memulai training...\n")
print("=" * 60)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "=" * 60)
print("✅ Training selesai!")

## 📊 Visualisasi Hasil Training

In [ ]:
# Plot training history
def plot_training_history(history):
    """Visualisasi training & validation metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Training Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Training Precision')
    axes[1, 0].plot(history.history['val_precision'], label='Validation Precision')
    axes[1, 0].set_title('Model Precision', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Training Recall')
    axes[1, 1].plot(history.history['val_recall'], label='Validation Recall')
    axes[1, 1].set_title('Model Recall', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

In [ ]:
# Tampilkan final metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print("📊 Final Training Metrics:")
print("=" * 60)
print(f"Training Accuracy:    {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"Validation Accuracy:  {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
print(f"Training Loss:        {final_train_loss:.4f}")
print(f"Validation Loss:      {final_val_loss:.4f}")
print("=" * 60)

# Evaluasi overfitting
overfit_gap = final_train_acc - final_val_acc
if overfit_gap > 0.1:
    print("\n⚠️ WARNING: Model mungkin overfitting (gap > 10%)")
    print("   Solusi: Tambah data augmentasi atau regularisasi")
elif overfit_gap < -0.05:
    print("\n⚠️ WARNING: Validation accuracy lebih tinggi dari training")
    print("   Ini tidak biasa, mungkin perlu cek dataset split")
else:
    print("\n✅ Model generalisasi dengan baik!")

## 💾 Simpan Model

Menyimpan model final dalam format Keras (.h5)

In [ ]:
# Simpan model final
print(f"💾 Menyimpan model ke: {MODEL_SAVE_PATH}")
model.save(MODEL_SAVE_PATH)
print("✅ Model berhasil disimpan!\n")

# Verifikasi file tersimpan
import os
if os.path.exists(MODEL_SAVE_PATH):
    file_size = os.path.getsize(MODEL_SAVE_PATH) / (1024 * 1024)  # MB
    print(f"📁 File Info:")
    print(f"   - Path: {MODEL_SAVE_PATH}")
    print(f"   - Size: {file_size:.2f} MB")
    print(f"   - Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("❌ ERROR: File tidak ditemukan!")

## 📥 Download Model

Download model untuk digunakan di Fase 2 & 3

In [ ]:
# Download model ke komputer lokal
from google.colab import files

print("📥 Mendownload model...")
files.download(MODEL_SAVE_PATH)
print("\n✅ Download selesai! File akan muncul di folder Downloads Anda.")

## 🧪 Test Model (Quick Verification)

Test cepat model dengan beberapa gambar dari validation set

In [ ]:
# Test model dengan sample images
def test_model_predictions(model, generator, num_samples=8):
    """Test model dengan sample gambar dan tampilkan prediksi"""
    # Ambil batch dari generator
    images, true_labels = next(generator)
    
    # Prediksi
    predictions = model.predict(images, verbose=0)
    
    # Reverse class indices
    reverse_indices = {v: k for k, v in generator.class_indices.items()}
    
    # Visualisasi
    plt.figure(figsize=(16, 8))
    for i in range(min(num_samples, len(images))):
        plt.subplot(2, 4, i + 1)
        
        # Denormalisasi gambar
        img = images[i]
        img = (img - img.min()) / (img.max() - img.min())
        
        # Label & prediksi
        true_label = reverse_indices[int(true_labels[i])]
        pred_prob = predictions[i][0]
        pred_label = reverse_indices[1] if pred_prob > 0.5 else reverse_indices[0]
        
        # Warna berdasarkan correctness
        color = 'green' if true_label == pred_label else 'red'
        
        plt.imshow(img)
        plt.title(
            f"True: {true_label}\nPred: {pred_label} ({pred_prob:.2%})",
            color=color,
            fontsize=10
        )
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Hitung accuracy
    predicted_classes = (predictions > 0.5).astype(int).flatten()
    true_classes = true_labels[:len(predicted_classes)].astype(int)
    accuracy = (predicted_classes == true_classes).mean()
    
    print(f"\n🎯 Sample Accuracy: {accuracy:.2%}")

print("🧪 Testing model dengan validation samples...\n")
test_model_predictions(model, validation_generator)

## ✅ Fase 1 Selesai!

### 📦 Output yang Dihasilkan:
1. **qc_inspector_model.h5** - Model Keras lengkap
2. **best_model_checkpoint.h5** - Model checkpoint terbaik

### 📝 Informasi Penting untuk Fase Selanjutnya:

**Class Indices (CATAT INI!):**
```python
# Akan ditampilkan di sel "Verifikasi Label" di atas
# Contoh: {'cacat_produksi': 0, 'lulus_qc': 1}
```

**Model Performance:**
- Validation Accuracy: [Lihat hasil di atas]
- Model Size: [Lihat hasil di atas]

### 🎯 Langkah Selanjutnya:
1. **Download model** (qc_inspector_model.h5)
2. **Fase 2:** Verifikasi model dengan Python script
3. **Fase 3:** Konversi model ke TFLite untuk deployment

### 💡 Tips:
- Jika accuracy < 85%, pertimbangkan:
  - Tambah jumlah EPOCHS
  - Tambah data training
  - Fine-tune hyperparameters
- Jika overfitting, tambah:
  - Data augmentasi
  - Dropout rate
  - Regularisasi

---

**Created with ❤️ for PCB Defect Detection**